In [1]:
import os
import pandas as pd
import json
from dotenv import load_dotenv

from neomodel import (
    StructuredNode,
    StringProperty,
    IntegerProperty,
    RelationshipTo,
)

load_dotenv()

True

In [2]:
URI=os.getenv("NEO4J_URI")
USERNAME=os.getenv("NEO4J_USERNAME")
PASSWORD=os.getenv("NEO4J_PASSWORD")

In [3]:
franchise_df = pd.read_excel("cafe_graph_data.xlsx", sheet_name="Franchise")
product_df = pd.read_excel("cafe_graph_data.xlsx", sheet_name="Product")
daerah_df = pd.read_excel("cafe_graph_data.xlsx", sheet_name="Daerah")
employee_df = pd.read_excel("cafe_graph_data.xlsx", sheet_name="Employee")

In [ ]:
class Daerah(StructuredNode):
    id_cafe = StringProperty()
    nama_daerah = StringProperty()
    kecamatan = StringProperty()
    kota = StringProperty()
    kode_pos = StringProperty()

class Franchise(StructuredNode):
    id_cafe = StringProperty(unique_index=True)
    name = StringProperty()
    year = IntegerProperty()

    # Relationships
    is_located = RelationshipTo("Daerah", "IS_LOCATED")
    has_product = RelationshipTo("Product", "HAS_PRODUCT")
    has_employee = RelationshipTo("Employee", "HAS_EMPLOYEE")

class Product(StructuredNode):
    id_product = StringProperty(unique_index=True)
    name = StringProperty()
    category = StringProperty()
    price = IntegerProperty()

class Employee(StructuredNode):
    id_employee = StringProperty(unique_index=True) 
    name = StringProperty()
    work_start_hour = StringProperty()
    work_end_hour = StringProperty()
    id_cafe = StringProperty()


In [ ]:
for _, row in daerah_df.iterrows():
    if not Daerah.nodes.get_or_none(id_cafe=row["id_cafe"]):
        Daerah(
            id_cafe=row["id_cafe"],
            nama_daerah=row["nama_daerah"],
            kecamatan=row["kecamatan"],
            kota=row["kota"],
            kode_pos=row["kode_pos"]
        ).save()

product_map = {}
for _, row in product_df.iterrows():
    required_columns = ["id", "name", "category", "price"]
    if all(col in product_df.columns for col in required_columns):
        product_id = row["id"]
        if not Product.nodes.get_or_none(id_product=product_id):
            product = Product(
                id_product=product_id,
                name=row["name"],
                category=row["category"],
                price=row["price"]
            ).save()
            product_map[product_id] = product
    else:
        print(f"Skipping invalid product row: {row}")

for _, row in franchise_df.iterrows():
    franchise_id = row["id_cafe"]
    if not Franchise.nodes.get_or_none(id_cafe=franchise_id):
        Franchise(
            id_cafe=franchise_id,
            name=row["cafe_name"],
            year=row["year_established"]
        ).save()

for _, row in employee_df.iterrows():
    employee_id = row["id"]
    if not Employee.nodes.get_or_none(id_employee=employee_id):
        Employee(
            id_employee=employee_id,
            name=row["name"],
            work_start_hour=row["work_start_hour"],
            work_end_hour=row["work_end_hour"],
            id_cafe=row["id_cafe"]
        ).save()



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Daerah)} {position: line: 1, column: 16, offset: 15} for query: ' MATCH (daerah:Daerah) WHERE daerah.id_cafe = $daerah_id_cafe_1 RETURN daerah LIMIT 2'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing propert

In [ ]:
for franchise in Franchise.nodes.all():
    daerah_node = Daerah.nodes.get(id_cafe=franchise.id_cafe)
    if not franchise.is_located.is_connected(daerah_node):
        franchise.is_located.connect(daerah_node)
    
    employees = Employee.nodes.filter(id_cafe=franchise.id_cafe)
    for emp in employees:
        if not franchise.has_employee.is_connected(emp):
            franchise.has_employee.connect(emp)

all_products = Product.nodes.all()
for franchise in Franchise.nodes.all():
    for prod in all_products:
        if not franchise.has_product.is_connected(prod):
            franchise.has_product.connect(prod)

print("Data imported successfully!")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: IS_LOCATED)} {position: line: 1, column: 96, offset: 95} for query: 'MATCH (franchise) WHERE elementId(franchise)=$franchise_1 WITH franchise MATCH (franchise)-[r1:`IS_LOCATED`]->(is_located_r1:Daerah) WHERE elementId(is_located_r1) = $is_located_r1_contains_1 WITH is_located_r1 RETURN count(is_located_r1)'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {descripti

Data imported successfully!
